In [24]:
import numpy as np
import operator
from IPython.display import clear_output
from time import sleep
import random

In [25]:
class envoirnment:
    def __init__(self,observation_space_n,action_space_n,state_space,reward_space,state_space_width,state_space_length):
        self.observation_space_n=observation_space_n
        self.action_space_n=action_space_n
        self.state_space=state_space
        self.reward_space=reward_space
        self.state_space_width=state_space_width
        self.state_space_length=state_space_length

    def __del__(self):
        pass

In [26]:
def create_random_policy(env):
    policy = {}
    for key in range(0, env.observation_space_n):
        current_end = 0
        p = {}
        for action in range(0, env.action_space_n):
            p[action] = 1 / env.action_space_n
        policy[key] = p
    return policy

In [27]:
def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
         Q[key] = {a: 0.0 for a in range(0, env.action_space_n)}
    return Q

In [28]:
def run_game(env, policy, display=True):
    # env.reset()
    state = env.state_space.index('S')
    episode = []
    finished = False

    while not finished:
        s = state

        timestep = []
        timestep.append(s)
        n = random.uniform(0, sum(policy[s].values()))
        top_range = 0
        for prob in policy[s].items():
                top_range += prob[1]
                if n < top_range:
                    action = prob[0]
                    break 

        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        state_x=state%env.state_space_width
        state_y=state//env.state_space_width
        if(action==0):
            if (state_y>0):
                state_y-=1
        elif(action==1):
            if (state_x<env.state_space_width-1):
                state_x+=1
        elif(action==2):
            if (state_y<env.state_space_length-1):
                state_y+=1
        elif(action==3):
            if (state_x>0):
                state_x-=1
        new_state=state_x+state_y*env.state_space_width
        reward=env.reward_space[new_state]
        if(env.state_space[new_state]=='G' or env.state_space[new_state]=='W'):
            finished=True
        else:
            finished=False
        state= new_state
        # state, reward, finished,terminated, info = env.step(action)
        timestep.append(action)
        timestep.append(reward)

        episode.append(timestep)
        # if display:
        #     # clear_output(True)
        #     print(state_x,state_y)
        #     # env.render()
        #     # sleep(1)

    # if display:
    #     # clear_output(True)
    #     # env.render()
    #     print('state_x,state_y')
    #     # sleep(1)
    return episode

In [29]:
def test_policy(policy, env,r=100):
    wins = 0
    episodes=[]
    for i in range(r):
        temp=run_game(env, policy, display=True)
        episodes.append(temp)
        w = temp[-1][-1]
        # print(w)
        if w == 1:
                wins += 1
    return episodes

In [30]:
def monte_carlo_e_soft(env, episodes=100, policy=None, epsilon=0.01):
    if not policy:
        policy = create_random_policy(env)  # Create an empty dictionary to store state action values    
    Q = create_state_action_dictionary(env, policy) # Empty dictionary for storing rewards for each state-action pair
    returns = {}
    
    for _ in range(episodes): # Looping through episodes
        G = 0 # Store cumulative reward in G (initialized at 0)
        episode = run_game(env=env, policy=policy, display=False) # Store state, action and value respectively 
        
        # for loop through reversed indices of episode array. 
        # The logic behind it being reversed is that the eventual reward would be at the end. 
        # So we have to go back from the last timestep to the first one propagating result from the future.
        
        for i in reversed(range(0, len(episode))):   
            s_t, a_t, r_t = episode[i] 
            state_action = (s_t, a_t)
            G += r_t # Increment total reward by reward on current timestep
            
            if not state_action in [(x[0], x[1]) for x in episode[0:i]]: # 
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]   
                    
                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                
                Q_list = list(map(lambda x: x[1], Q[s_t].items())) # Finding the action with maximum value
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                max_Q = random.choice(indices)
                
                A_star = max_Q
                
                for a in policy[s_t].items(): # Update action probability for s_t in policy
                    if a[0] == A_star:
                        policy[s_t][a[0]] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                    else:
                        policy[s_t][a[0]] = (epsilon / abs(sum(policy[s_t].values())))

    return policy

In [31]:
# env = gym.make('FrozenLake8x8-v1')
action_space = [0,1,2,3]#up,right,down,left,nothing
state_space_width=20 # S is start, T is tile, W is Wall, G is Goal
state_space_length=20
state_space = ['T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','W',
               'T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','G',
               'T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','W','T','T',
               'T','T','T','T','T','T','T','W','W','T','W','T','T','T','T','T','W','T','T','T',
               'T','T','T','T','T','T','W','T','T','T','W','T','T','T','T','W','T','T','T','T',
               'T','T','T','T','T','T','W','T','W','T','W','T','T','T','W','T','T','T','T','T',
               'T','T','T','T','T','T','W','T','W','T','W','T','T','W','T','T','T','T','T','T',
               'T','W','T','T','T','T','W','T','W','T','W','T','W','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','W','T','T','W','T','T','T','T','T','T','T','T',
               'S','W','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T']
state_space_width=5 # S is start, T is tile, W is Wall, G is Goal
state_space_length=5
state_space = ['S','T','T','T','T',
               'T','T','T','T','T',
               'T','W','W','T','T',
               'T','T','T','T','T',
               'T','T','T','T','G']
# for i in range(len(state_space)):
#     if not (state_space[i]=='S' or state_space[i]=='G'):
#         if(random.randint(0,8)==0):
#             state_space[i]='W'
#         else:
#             state_space[i]='T'
# print(len(state_space))
reward_space = []
for i in state_space:
    if(i=='S'):
        reward_space.append(-1)
    elif(i=='G'):
        reward_space.append(10)
    elif(i=='T'):
        reward_space.append(-1)
    elif(i=='W'):
        reward_space.append(-100)
# print('  Reward matrix')
# for i in range(len(reward_space)):
#     print('{:4}'.format(reward_space[i]),end = ',')
#     if(i%state_space_width==state_space_width-1):
#         print()

observation_space_n=len(state_space)
action_space_n=len(action_space)
state_space
reward_space
state_space_width
state_space_length
env=envoirnment(observation_space_n,action_space_n,state_space,reward_space,state_space_width,state_space_length)
policy = monte_carlo_e_soft(env, episodes=10000)
# for i in policy:
#     print(i,policy[i])

In [32]:
episodes=test_policy(policy, env,1)
for j in episodes:
    steps_taken=[]
    for i in j:
        # print(i[0]%state_space_width,i[0]//state_space_width)
        steps_taken.append(i[0])

    for i in range(len(state_space)):
        if(i not in steps_taken):
            print('{:3}'.format(state_space[i]),end = ' ')
        else:
            if(state_space[i] == 'G'):
                print('{:3}'.format('A_G'),end = ' ')
            else:
                print('{:3}'.format('A'),end = ' ')
        if(i%state_space_width==state_space_width-1):
            print()
    # We print the number of step it took.
    print("Number of steps", len(steps_taken))

A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   T   
T   T   T   A   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   T   
T   T   T   A   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   T   
T   T   T   A   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   T   
T   T   T   A   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   T   
T   T   T   A   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   T   
T   T   T   A   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   A   
T   T   T   T   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   T   
T   T   T   A   T   
T   W   W   A   T   
T   T   T   A   A   
T   T   T   T   G   
Number of steps 8
A   A   A   A   